## TEDS DATASET

## Import Libraries
Load pandas for data manipulation, numpy for numerical operations

In [ ]:
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm

warnings.filterwarnings("ignore")

## Load Data
Read the raw TEDS-A 2023 CSV file into a pandas DataFrame

In [ ]:
def load_data(filepath, chunksize=100000):
    chunks = []
    for chunk in pd.read_csv(filepath, chunksize=chunksize, low_memory=False):
        chunks.append(chunk)
    df = pd.concat(chunks, ignore_index=True)
    return df

## Handle Missing Values
Replace TEDS standard missing value code (-9) with NaN for proper handling

In [ ]:
def handle_missing_values(df):
    df = df.replace(-9, np.nan)
    return df

## Optimize Data Types
Convert columns to appropriate data types (Int8 for binary, category for categorical, Int64 for IDs) to reduce memory usage

In [ ]:
def optimize_datatypes(df):
    binary_cols = [
        "IDU",
        "ALCFLG",
        "COKEFLG",
        "MARFLG",
        "HERFLG",
        "METHFLG",
        "OPSYNFLG",
        "PCPFLG",
        "HALLFLG",
        "MTHAMFLG",
        "AMPHFLG",
        "STIMFLG",
        "BENZFLG",
        "TRNQFLG",
        "BARBFLG",
        "SEDHPFLG",
        "INHFLG",
        "OTCFLG",
        "OTHERFLG",
    ]

    categorical_cols = [
        "STFIPS",
        "REGION",
        "DIVISION",
        "EDUC",
        "MARSTAT",
        "SERVICES",
        "DETCRIM",
        "NOPRIOR",
        "PSOURCE",
        "EMPLOY",
        "METHUSE",
        "PSYPROB",
        "SEX",
        "VET",
        "PREG",
        "LIVARAG",
        "DSMCRIT",
        "RACE",
        "ETHNIC",
        "DETNLF",
        "PRIMINC",
        "SUB1",
        "SUB2",
        "SUB3",
        "ROUTE1",
        "ROUTE2",
        "ROUTE3",
        "FREQ1",
        "FREQ2",
        "FREQ3",
        "FRSTUSE1",
        "FRSTUSE2",
        "FRSTUSE3",
        "HLTHINS",
        "PRIMPAY",
        "FREQ_ATND_SELF_HELP",
        "ALCDRUG",
        "DAYWAIT",
        "AGE",
        "ARRESTS",
    ]

    int_cols = ["ADMYR", "CASEID", "CBSA2020"]

    for col in binary_cols:
        if col in df.columns:
            df[col] = df[col].astype("Int8")

    for col in categorical_cols:
        if col in df.columns:
            df[col] = df[col].astype("category")

    for col in int_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

    return df

## Engineer Features
Create 17 new features for treatment optimization: substance use duration, polysubstance indicators, risk factors, vulnerable populations, and social determinants


In [ ]:
def engineer_features(df):
    def to_num(series):
        return pd.to_numeric(series, errors="coerce").fillna(0)

    age_midpoints = {
        1: 13,
        2: 16,
        3: 19,
        4: 22.5,
        5: 27,
        6: 32,
        7: 37,
        8: 42,
        9: 47,
        10: 52,
        11: 59.5,
        12: 70,
    }
    frstuse_midpoints = {1: 11, 2: 13, 3: 16, 4: 19, 5: 22.5, 6: 27, 7: 40}

    if "AGE" in df.columns and "FRSTUSE1" in df.columns:
        df["AGE_NUMERIC"] = pd.Series(df["AGE"]).map(age_midpoints)
        df["FRSTUSE1_NUMERIC"] = pd.Series(df["FRSTUSE1"]).map(frstuse_midpoints)
        df["AGE_NUMERIC"] = pd.to_numeric(df["AGE_NUMERIC"], errors="coerce")
        df["FRSTUSE1_NUMERIC"] = pd.to_numeric(df["FRSTUSE1_NUMERIC"], errors="coerce")
        df["YEARS_SINCE_FIRST_USE"] = (df["AGE_NUMERIC"] - df["FRSTUSE1_NUMERIC"]).clip(
            lower=0
        )

    if all(col in df.columns for col in ["SUB1", "SUB2", "SUB3"]):
        df["NUM_SUBSTANCES"] = 0
        df.loc[
            (pd.Series(df["SUB1"]).notna()) & (pd.Series(df["SUB1"]) != 1),
            "NUM_SUBSTANCES",
        ] += 1
        df.loc[
            (pd.Series(df["SUB2"]).notna()) & (pd.Series(df["SUB2"]) != 1),
            "NUM_SUBSTANCES",
        ] += 1
        df.loc[
            (pd.Series(df["SUB3"]).notna()) & (pd.Series(df["SUB3"]) != 1),
            "NUM_SUBSTANCES",
        ] += 1
        df["POLYSUBSTANCE_USER"] = (df["NUM_SUBSTANCES"] >= 2).astype(int)

    if "SUB1" in df.columns:
        df["ALCOHOL_PRIMARY"] = (to_num(df["SUB1"]) == 1).astype(int)
        df["OPIOID_PRIMARY"] = to_num(df["SUB1"]).isin([5, 6, 7]).astype(int)
        df["STIMULANT_PRIMARY"] = to_num(df["SUB1"]).isin([3, 10, 11, 12]).astype(int)

    if "IDU" in df.columns:
        df["INJECTION_USER"] = np.where(to_num(df["IDU"]) == 1, 1, 0)

    if "PSOURCE" in df.columns:
        df["CJ_REFERRAL"] = (to_num(df["PSOURCE"]) == 7).astype(int)

    if "ARRESTS" in df.columns:
        df["RECENT_ARREST"] = np.where(to_num(df["ARRESTS"]) > 0, 1, 0)

    if "NOPRIOR" in df.columns:
        df["CHRONIC_TREATMENT"] = np.where(to_num(df["NOPRIOR"]) >= 3, 1, 0)
        df["TREATMENT_NAIVE"] = np.where(to_num(df["NOPRIOR"]) == 0, 1, 0)

    if "DAYWAIT" in df.columns:
        df["DAYWAIT"] = pd.to_numeric(df["DAYWAIT"], errors="coerce")
        df["LONG_WAIT"] = np.where(to_num(df["DAYWAIT"]) >= 3, 1, 0)

    if "AGE" in df.columns:
        df["ADOLESCENT"] = (to_num(df["AGE"]).isin([1, 2])).astype(int)
        df["OLDER_ADULT"] = (to_num(df["AGE"]).isin([11, 12])).astype(int)

    if "PREG" in df.columns:
        df["PREGNANT"] = (df["PREG"] == 1).astype(int)

    if "LIVARAG" in df.columns:
        df["HOMELESS"] = (df["LIVARAG"] == 1).astype(int)

    if "PRIMINC" in df.columns:
        df["NO_INCOME"] = (df["PRIMINC"] == 5).astype(int)

    if "PSYPROB" in df.columns:
        df["HAS_MENTAL_HEALTH"] = (df["PSYPROB"] == 1).astype(int)

    if "SUB1" in df.columns:
        substance_groups = {
            "Alcohol": [2],
            "Opioids": [5, 6, 7],
            "Stimulants": [3, 10, 11, 12],
            "Cannabis": [4],
            "Sedatives": [13, 14, 15, 16],
            "Other": [8, 9, 17, 18, 19],
            "None": [1],
        }

        def categorize_substance(sub):
            if pd.isna(sub):
                return "Unknown"
            for group, codes in substance_groups.items():
                if sub in codes:
                    return group
            return "Other"

        df["PRIMARY_SUB_CATEGORY"] = df["SUB1"].apply(categorize_substance)

    return df

## Decode Categorical Values
Convert numeric codes to readable text labels (e.g., AGE: 7 → '35-39', SEX: 1 → 'Male') for better interpretability in EDA


In [ ]:
def decode_categorical_values(df):
    age_labels = {
        1: "12-14",
        2: "15-17",
        3: "18-20",
        4: "21-24",
        5: "25-29",
        6: "30-34",
        7: "35-39",
        8: "40-44",
        9: "45-49",
        10: "50-54",
        11: "55-64",
        12: "65+",
    }

    sex_labels = {1: "Male", 2: "Female"}

    race_labels = {
        1: "Alaska Native",
        2: "American Indian",
        3: "Asian/Pacific Islander",
        4: "Black/African American",
        5: "White",
        6: "Asian",
        7: "Other single race",
        8: "Two or more races",
        9: "Native Hawaiian/Other Pacific Islander",
    }

    ethnic_labels = {
        1: "Puerto Rican",
        2: "Mexican",
        3: "Cuban/Other Hispanic",
        4: "Not Hispanic/Latino",
        5: "Hispanic, unspecified",
    }

    marstat_labels = {
        1: "Never married",
        2: "Now married",
        3: "Separated",
        4: "Divorced/widowed",
    }

    educ_labels = {
        1: "Less than Grade 9",
        2: "Grades 9-11",
        3: "Grade 12/GED",
        4: "1-3 years college",
        5: "4+ years college",
    }

    employ_labels = {
        1: "Full-time",
        2: "Part-time",
        3: "Unemployed",
        4: "Not in labor force",
    }

    livarag_labels = {1: "Homeless", 2: "Dependent living", 3: "Independent living"}

    priminc_labels = {
        1: "Wages/salary",
        2: "Public assistance",
        3: "Retirement/pension/disability",
        4: "Other",
        5: "None",
    }

    arrests_labels = {0: "None", 1: "Once", 2: "Two or more"}

    services_labels = {
        1: "Detox 24hr Hospital",
        2: "Detox 24hr Residential",
        3: "Rehab Hospital",
        4: "Rehab Short-term (≤30 days)",
        5: "Rehab Long-term (>30 days)",
        6: "Intensive Outpatient",
        7: "Non-intensive Outpatient",
        8: "Outpatient Detox",
    }

    daywait_labels = {
        0: "Same day",
        1: "1-7 days",
        2: "8-14 days",
        3: "15-30 days",
        4: "31+ days",
    }

    psource_labels = {
        1: "Self/Individual",
        2: "Alcohol/drug provider",
        3: "Other healthcare",
        4: "School",
        5: "Employer/EAP",
        6: "Other community",
        7: "Criminal justice/DUI/DWI",
    }

    noprior_labels = {
        0: "No prior",
        1: "One prior",
        2: "Two prior",
        3: "Three prior",
        4: "Four prior",
        5: "Five or more prior",
    }

    substance_labels = {
        1: "None",
        2: "Alcohol",
        3: "Cocaine/crack",
        4: "Marijuana/hashish",
        5: "Heroin",
        6: "Non-Rx methadone",
        7: "Other opiates/synthetics",
        8: "PCP",
        9: "Hallucinogens",
        10: "Methamphetamine/speed",
        11: "Other amphetamines",
        12: "Other stimulants",
        13: "Benzodiazepines",
        14: "Other tranquilizers",
        15: "Barbiturates",
        16: "Other sedatives/hypnotics",
        17: "Inhalants",
        18: "OTC medications",
        19: "Other drugs",
    }

    route_labels = {
        1: "Oral",
        2: "Smoking",
        3: "Inhalation",
        4: "Injection",
        5: "Other",
    }

    freq_labels = {1: "No use past month", 2: "Some use", 3: "Daily use"}

    frstuse_labels = {
        1: "11 and under",
        2: "12-14",
        3: "15-17",
        4: "18-20",
        5: "21-24",
        6: "25-29",
        7: "30+",
    }

    methuse_labels = {1: "Yes", 2: "No"}

    psyprob_labels = {1: "Yes", 2: "No"}

    hlthins_labels = {
        1: "Private/Blue Cross/HMO",
        2: "Medicaid",
        3: "Medicare/Other",
        4: "None",
    }

    primpay_labels = {
        1: "Self-pay",
        2: "Private insurance",
        3: "Medicare",
        4: "Medicaid",
        5: "Other government",
        6: "No charge",
        7: "Other",
    }

    freq_selfhelp_labels = {
        1: "No attendance",
        2: "1-3 times/month",
        3: "4-7 times/month",
        4: "8-30 times/month",
        5: "Some attendance, frequency unknown",
    }

    idu_labels = {0: "Not reported", 1: "IDU reported"}

    preg_labels = {1: "Yes", 2: "No"}

    vet_labels = {1: "Yes", 2: "No"}

    region_labels = {
        0: "US territories",
        1: "Northeast",
        2: "Midwest",
        3: "South",
        4: "West",
    }

    if "AGE" in df.columns:
        df["AGE"] = pd.Series(df["AGE"]).map(age_labels)

    if "SEX" in df.columns:
        df["SEX"] = pd.Series(df["SEX"]).map(sex_labels)

    if "RACE" in df.columns:
        df["RACE"] = pd.Series(df["RACE"]).map(race_labels)

    if "ETHNIC" in df.columns:
        df["ETHNIC"] = pd.Series(df["ETHNIC"]).map(ethnic_labels)

    if "MARSTAT" in df.columns:
        df["MARSTAT"] = pd.Series(df["MARSTAT"]).map(marstat_labels)

    if "EDUC" in df.columns:
        df["EDUC"] = pd.Series(df["EDUC"]).map(educ_labels)

    if "EMPLOY" in df.columns:
        df["EMPLOY"] = pd.Series(df["EMPLOY"]).map(employ_labels)

    if "LIVARAG" in df.columns:
        df["LIVARAG"] = pd.Series(df["LIVARAG"]).map(livarag_labels)

    if "PRIMINC" in df.columns:
        df["PRIMINC"] = pd.Series(df["PRIMINC"]).map(priminc_labels)

    if "ARRESTS" in df.columns:
        df["ARRESTS"] = pd.Series(df["ARRESTS"]).map(arrests_labels)

    if "SERVICES" in df.columns:
        df["SERVICES"] = pd.Series(df["SERVICES"]).map(services_labels)

    if "DAYWAIT" in df.columns:
        df["DAYWAIT"] = pd.Series(df["DAYWAIT"]).map(daywait_labels)

    if "PSOURCE" in df.columns:
        df["PSOURCE"] = pd.Series(df["PSOURCE"]).map(psource_labels)

    if "NOPRIOR" in df.columns:
        df["NOPRIOR"] = pd.Series(df["NOPRIOR"]).map(noprior_labels)

    if "SUB1" in df.columns:
        df["SUB1"] = pd.Series(df["SUB1"]).map(substance_labels)

    if "SUB2" in df.columns:
        df["SUB2"] = pd.Series(df["SUB2"]).map(substance_labels)

    if "SUB3" in df.columns:
        df["SUB3"] = pd.Series(df["SUB3"]).map(substance_labels)

    if "ROUTE1" in df.columns:
        df["ROUTE1"] = pd.Series(df["ROUTE1"]).map(route_labels)

    if "ROUTE2" in df.columns:
        df["ROUTE2"] = pd.Series(df["ROUTE2"]).map(route_labels)

    if "ROUTE3" in df.columns:
        df["ROUTE3"] = pd.Series(df["ROUTE3"]).map(route_labels)

    if "FREQ1" in df.columns:
        df["FREQ1"] = pd.Series(df["FREQ1"]).map(freq_labels)

    if "FREQ2" in df.columns:
        df["FREQ2"] = pd.Series(df["FREQ2"]).map(freq_labels)

    if "FREQ3" in df.columns:
        df["FREQ3"] = pd.Series(df["FREQ3"]).map(freq_labels)

    if "FRSTUSE1" in df.columns:
        df["FRSTUSE1"] = pd.Series(df["FRSTUSE1"]).map(frstuse_labels)

    if "METHUSE" in df.columns:
        df["METHUSE"] = pd.Series(df["METHUSE"]).map(methuse_labels)

    if "PSYPROB" in df.columns:
        df["PSYPROB"] = pd.Series(df["PSYPROB"]).map(psyprob_labels)

    if "HLTHINS" in df.columns:
        df["HLTHINS"] = pd.Series(df["HLTHINS"]).map(hlthins_labels)

    if "PRIMPAY" in df.columns:
        df["PRIMPAY"] = pd.Series(df["PRIMPAY"]).map(primpay_labels)

    if "FREQ_ATND_SELF_HELP" in df.columns:
        df["FREQ_ATND_SELF_HELP"] = pd.Series(df["FREQ_ATND_SELF_HELP"]).map(
            freq_selfhelp_labels
        )

    if "IDU" in df.columns:
        df["IDU"] = pd.Series(df["IDU"]).map(idu_labels)

    if "PREG" in df.columns:
        df["PREG"] = pd.Series(df["PREG"]).map(preg_labels)

    if "VET" in df.columns:
        df["VET"] = pd.Series(df["VET"]).map(vet_labels)

    if "REGION" in df.columns:
        df["REGION"] = pd.Series(df["REGION"]).map(region_labels)

    return df

## Select and Rename Columns
Keep only 50 relevant columns for treatment optimization and rename them to human-readable names (e.g., CASEID → patient_id, SERVICES → service_type)


In [ ]:
def select_and_rename_columns(df):
    column_mapping = {
        "CASEID": "patient_id",
        "AGE": "age_group",
        "SEX": "sex",
        "RACE": "race",
        "ETHNIC": "ethnicity",
        "MARSTAT": "marital_status",
        "EDUC": "education_level",
        "EMPLOY": "employment_status",
        "LIVARAG": "living_arrangement",
        "PRIMINC": "income_source",
        "ARRESTS": "recent_arrests",
        "SERVICES": "service_type",
        "DAYWAIT": "wait_time_days",
        "PSOURCE": "referral_source",
        "NOPRIOR": "prior_treatments",
        "SUB1": "primary_substance",
        "SUB2": "secondary_substance",
        "SUB3": "tertiary_substance",
        "ROUTE1": "route_primary",
        "ROUTE2": "route_secondary",
        "ROUTE3": "route_tertiary",
        "FREQ1": "frequency_primary",
        "FREQ2": "frequency_secondary",
        "FREQ3": "frequency_tertiary",
        "FRSTUSE1": "age_first_use_primary",
        "METHUSE": "medication_assisted_therapy",
        "DSMCRIT": "dsm_diagnosis",
        "PSYPROB": "has_cooccurring_mental_health",
        "HLTHINS": "health_insurance",
        "PRIMPAY": "payment_source",
        "FREQ_ATND_SELF_HELP": "self_help_attendance",
        "IDU": "injection_drug_use",
        "PREG": "pregnant",
        "VET": "veteran_status",
        "STFIPS": "state",
        "REGION": "region",
        "YEARS_SINCE_FIRST_USE": "years_using",
        "NUM_SUBSTANCES": "number_of_substances",
        "POLYSUBSTANCE_USER": "is_polysubstance",
        "OPIOID_PRIMARY": "is_opioid_primary",
        "STIMULANT_PRIMARY": "is_stimulant_primary",
        "INJECTION_USER": "is_injection_user",
        "CJ_REFERRAL": "is_criminal_justice_referral",
        "RECENT_ARREST": "has_recent_arrest",
        "CHRONIC_TREATMENT": "is_chronic_treatment",
        "TREATMENT_NAIVE": "is_first_treatment",
        "LONG_WAIT": "has_long_wait",
        "ADOLESCENT": "is_adolescent",
        "OLDER_ADULT": "is_older_adult",
        "PREGNANT": "is_pregnant",
        "HOMELESS": "is_homeless",
        "NO_INCOME": "has_no_income",
        "HAS_MENTAL_HEALTH": "has_mental_health_disorder",
        "PRIMARY_SUB_CATEGORY": "substance_category",
    }

    relevant_columns = [
        "patient_id",
        "age_group",
        "sex",
        "race",
        "ethnicity",
        "marital_status",
        "education_level",
        "employment_status",
        "living_arrangement",
        "income_source",
        "recent_arrests",
        "service_type",
        "wait_time_days",
        "referral_source",
        "prior_treatments",
        "primary_substance",
        "secondary_substance",
        "tertiary_substance",
        "route_primary",
        "frequency_primary",
        "age_first_use_primary",
        "medication_assisted_therapy",
        "dsm_diagnosis",
        "has_cooccurring_mental_health",
        "health_insurance",
        "payment_source",
        "self_help_attendance",
        "injection_drug_use",
        "pregnant",
        "veteran_status",
        "state",
        "region",
        "years_using",
        "number_of_substances",
        "is_polysubstance",
        "is_opioid_primary",
        "is_stimulant_primary",
        "is_injection_user",
        "is_criminal_justice_referral",
        "has_recent_arrest",
        "is_chronic_treatment",
        "is_first_treatment",
        "has_long_wait",
        "is_adolescent",
        "is_older_adult",
        "is_pregnant",
        "is_homeless",
        "has_no_income",
        "has_mental_health_disorder",
        "substance_category",
    ]

    available_old_cols = [old for old in column_mapping.keys() if old in df.columns]
    df_renamed = df[available_old_cols].rename(columns=column_mapping)

    final_columns = [col for col in relevant_columns if col in df_renamed.columns]
    df_final = df_renamed[final_columns]

    return df_final

## Save Cleaned Data
Export the cleaned DataFrame to CSV file

In [ ]:
def save_cleaned_data(df, output_path):
    df.to_csv(output_path, index=False)
    return df

 ## Main Pipeline
Execute complete data cleaning workflow: load → handle missing → optimize types → engineer features → decode values → select/rename → save


In [ ]:
def clean_teds_data(input_filepath, output_filepath, chunksize=100000):
    first_chunk = True

    for chunk in tqdm(
        pd.read_csv(input_filepath, chunksize=chunksize, low_memory=False),
        desc="Cleaning chunks",
    ):
        chunk = handle_missing_values(chunk)
        chunk = optimize_datatypes(chunk)
        chunk = engineer_features(chunk)
        chunk = decode_categorical_values(chunk)
        chunk = select_and_rename_columns(chunk)

        mode = "w" if first_chunk else "a"
        header = first_chunk
        chunk.to_csv(output_filepath, index=False, mode=mode, header=header)
        first_chunk = False

## Execute Pipeline
Run the cleaning pipeline on TEDS-A 2023 data

In [ ]:
if __name__ == "__main__":
    INPUT_FILE = "1_datasets/raw/tedsa_puf_2023.csv"
    OUTPUT_FILE = "1_datasets/processed/teds_a_2023_cleaned.csv"

    df_cleaned = clean_teds_data(INPUT_FILE, OUTPUT_FILE)

## Make a Sample Dataset
Make a sample dataset for demonstration purposes

In [ ]:
df = pd.read_csv("1_datasets/processed/teds_a_2023_cleaned.csv")

df_sample = df.sample(1000, random_state=42)

df_sample.to_csv("1_datasets/sample/tedsa_sample.csv", index=False)